In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
import json

Automatyczny wybór funkcji, koncepcja Agentów i Single Entry Point, czyli jeden punkt wejścia.

In [2]:
# definicja schematów funkcji mnożenia, dodawania i odejmowania
multiplySchema = {
    "name": "multiply",
    "description": "Multiply two numbers",
    "parameters": {
        "type": "object",
        "properties": {
            "first": {
                "type": "number",
                "description": "First value to multiply"
            },
            "second": {
                "type": "number",
                "description": "Second value to multiply"
            }
        },
        "required": [
            "first", "second"
        ]
    }
}

addSchema = {
    "name": "add",
    "description": "Add two numbers",
    "parameters": {
        "type": "object",
        "properties": {
            "first": {
                "type": "number",
                "description": "First value to add"
            },
            "second": {
                "type": "number",
                "description": "Second value to add"
            }
        },
        "required": [
            "first", "second"
        ]
    }
}

subtractSchema = {
    "name": "subtract",
    "description": "Subtract two numbers",
    "parameters": {
        "type": "object",
        "properties": {
            "first": {
                "type": "number",
                "description": "First value to subtract"
            },
            "second": {
                "type": "number",
                "description": "Second value to subtract"
            }
        },
        "required": [
            "first", "second"
        ]
    }
}

In [3]:
# funkcja do parsowania nazwy funkcji wybranej przez model
def parse_function_call(result):
    try:
        result.additional_kwargs['function_call']
        function_call = result.additional_kwargs['function_call']
        return {
            "name": function_call["name"],
            "args": json.loads(function_call["arguments"])
        }
    except:
        return None

In [4]:
# zesatw narzędzi - dostępnych funkcji
tools = {
    "multiply": lambda a, b: a*b,
    "add": lambda a, b: a+b,
    "subtract": lambda a, b: a-b
}

In [5]:
model = ChatOpenAI(
    model = "gpt-4-0613"
).bind(
    functions= [multiplySchema, addSchema, subtractSchema],
    #function_call= {"name": "query_enrichment"},
)

result = model.invoke([
    HumanMessage("2929590 * 129359")
])

# Analizuj wywołanie funkcji z wyniku
action = parse_function_call(result)

# Wyświetl akcję, jeśli jest dostępna
if action:
    print(action["name"], action["args"])
    result = tools[action["name"]](action["args"]["first"], action["args"]["second"])
    print(result)
else:
    print("Brak wywołania funkcji w wyniku.")

multiply {'first': 2929590, 'second': 129359}
378968832810
